<a href="https://colab.research.google.com/github/rashmic20/Phase5_Project/blob/main/TopicModeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Topic modeling using LDA on Yelp review data.

Load the dataset and text preprocessing for LDA

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, SpatialDropout1D
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# For text preprocessing
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

In [4]:
data = pd.read_csv('restaurant_reviews_cleaned.csv')

# Display the first few rows
print(data.head())

# Assuming the text reviews are in a column named 'review_text'
reviews = data['cleaned_text'].dropna()

                review_id                 user_id             business_id  \
0  jA1JQhMMgAjS6KPUmeYUwg  xIqVxrEoHRx2eMeIC4HIEQ  c-IgS6Pk6vMyax7Rbr38eA   
1  0MwFRkSZShRfeBFWN1YzsQ  qkmzH5oT1V8_1GpBc3_7zw  tVxL-Ygzwp8NMuxwvNmWRA   
2  U8_ehvGLQzTK9drgHKObbA  OozcBexcgIKztqL49n8KxQ  _k8qEyZjGsGaCAtN_5EYSw   
3  KBBl_3h9xJ82kIAgdYtMCA  PEvrhKgW923jPcoB48MQAg  wJmyu7W1K9A_gE8Ed4Bc9w   
4  CE-CblTswQgRnoWSlMmJ4g  _7NoTSc5pyW8MzKNR0VKTQ  NLugmrrjRqw8jQGvard-eA   

   stars_x                 date                        name          city  \
0      1.0  2019-10-01 23:16:46        Barbecue and Bourbon      Speedway   
1      2.0  2019-11-11 23:13:16               Los Potrillos  Philadelphia   
2      5.0  2019-09-30 16:14:24      Tropical Smoothie Cafe          Lutz   
3      5.0  2019-10-19 19:08:30               Trident Grill        Tucson   
4      5.0  2019-10-04 23:15:49  Certified Kitchen + Bakery         Boise   

  state postal_code   latitude   longitude  stars_y  review_count  \
0    

Text Preprocessing

In [5]:
# Define a function to preprocess text
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove punctuation and numbers
    text = re.sub(r'\W+', ' ', text)
    text = re.sub(r'\d+', '', text)
    # Tokenize the text
    tokens = word_tokenize(text)
    # Remove stopwords
    tokens = [word for word in tokens if word not in stopwords.words('english')]
    # Lemmatize the tokens
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    # Join tokens back into a single string
    return ' '.join(tokens)

# Apply the preprocessing to the reviews
reviews_cleaned = reviews.apply(preprocess_text)

Vectorization

In [9]:
# Tokenize and pad sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
tokenizer = Tokenizer(num_words=5000, oov_token='<OOV>')
tokenizer.fit_on_texts(reviews_cleaned)
sequences = tokenizer.texts_to_sequences(reviews_cleaned)
padded_sequences = pad_sequences(sequences, maxlen=200, padding='post', truncating='post')

In [22]:
# Use CountVectorizer for LDA
vectorizer = CountVectorizer(max_df=0.95, min_df=2, stop_words='english')
X = vectorizer.fit_transform(reviews_cleaned)

In [23]:
# Define the LDA model
lda = LatentDirichletAllocation(n_components=10, random_state=42)

# Fit the model
lda.fit(X)

# Display the top words per topic
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print(f"Topic {topic_idx+1}:")
        print(" ".join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]))

no_top_words = 10
display_topics(lda, vectorizer.get_feature_names_out(), no_top_words)

Topic 1:
food great service place amazing friendly staff best delicious time
Topic 2:
order food time minute service said asked customer told didnt
Topic 3:
cheese fry sandwich good chicken burger sauce like ordered got
Topic 4:
taco chicken food good mexican place salsa burrito chip ordered
Topic 5:
pizza good roll place sushi food ordered chicken rice order
Topic 6:
place bar beer great good like food area drink lot
Topic 7:
good food coffee breakfast place really like egg sandwich time
Topic 8:
table mask covid restaurant room reservation dining staff experience party
Topic 9:
drink good cake cream got great delicious ice oyster really
Topic 10:
salad good dish delicious ordered meal restaurant dinner great steak


In [29]:
!pip install pyLDAvis

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 32.3 MB/s eta 0:00:00


In [31]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
import gensim

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [32]:
# Prepare the visualization
lda_vis = gensimvis.prepare(lda_model, corpus, dictionary)

# Display the visualization in a notebook (if using Jupyter)
pyLDAvis.display(lda_vis)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
